In [2]:
from pathlib import Path
DATA_DIR = Path.cwd().parent / 'data'

from tqdm.auto import tqdm # progress bars
import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

# Huggingface
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from datasets import Dataset, DatasetDict
from transformers import TrainingArguments, Trainer

MODEL_NAME = 'roberta-base'
TOKENIZER = RobertaTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

# Sklearn
from sklearn.metrics import precision_recall_fscore_support

# PyTorch
import torch
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(DEVICE)

cuda


In [30]:
def tokenize(batch):
    # Turns text into a series of numbers.
    return TOKENIZER(batch['text'],
                     padding="max_length",
                     truncation=True,
                     max_length=TOKENIZER.max_model_input_sizes[MODEL_NAME])

def columns_to_device(dataset):
    columns = ['input_ids', 'token_type_ids', 'attention_mask', 'label']
    columns = list(set(columns).intersection(dataset.column_names))
    dataset.set_format(type='torch', columns=columns, device=DEVICE)

def make_dataset(df):
    '''
    Takes a pandas dataframe and optionally prepocesses it a bit.
    Will return a Huggingface Dataset object.
    '''
    
    df['label'] = df['l1'].map(l1_codes)
    
    # Let's manage our data with Huggingface's Dataset class
    dataset = Dataset.from_pandas(df.reset_index(drop=True))
    dataset = dataset.map(tokenize, batched=True, num_proc=4)
    # dataset = dataset.remove_columns('text')
    
    columns_to_device(dataset)

    return dataset

In [31]:
# raw text csvs with metadata
train_path = DATA_DIR / 'efcamdat_train.parquet'
dev_path = DATA_DIR / 'efcamdat_dev.parquet'
test_path = DATA_DIR / 'efcamdat_test.parquet'

def read_data(path):
    return (pd
            .read_parquet(path)
            .loc[ :, [
                'writing_id',
                'learner_id',
                'l1',
                'cefr',
                'grade',
                'text',
            ]])

train_df = read_data(train_path)
dev_df = read_data(dev_path)
test_df = read_data(test_path)

l1_codes = {
    'Portuguese': 0,
    'Mandarin': 1,
    'Spanish': 2,
    'Russian': 3,
    'German': 4,
    'Arabic' : 5,
}

datadict = DatasetDict(
    {
        'train': make_dataset(train_df),
        'dev': make_dataset(dev_df),
        'test': make_dataset(test_df),
    }
)

In [34]:
from evaluate import evaluator
task_evaluator = evaluator("text-classification")
eval_results = task_evaluator.compute(
    model_or_pipeline="lvwerra/distilbert-imdb",
    data=data,
    metric=evaluate.combine(["accuracy", "recall", "precision", "f1"]),
    label_mapping=l1_codes
)

ModuleNotFoundError: No module named 'evaluate'

In [32]:
def model_init():
    return RobertaForSequenceClassification.from_pretrained(MODEL_NAME,
                                                            num_labels=6,
                                                           )

def compute_metrics(pred):
    metrics = ['precision', 'recall', 'f1', 'support']
    results = precision_recall_fscore_support(pred.label_ids,
                                              pred.predictions,
                                              labels=list(l1_codes.values()),
                                              zero_division=0,
                                             )
    return {k: v for k, v in zip(metrics, results)}



     
args = TrainingArguments(
    output_dir=Path.cwd().parent / 'bin',
    evaluation_strategy='epoch',
    learning_rate=3e-05, # Finetuned
    weight_decay=0.01, # Devlin et al. suggested
    num_train_epochs=3, # Finetuned
    seed=42, # apriori
    logging_strategy='epoch',
    save_strategy='no',
    fp16=True,
    load_best_model_at_end=False,
)

trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=datadict['train'],
    eval_dataset=datadict['dev'],
    tokenizer=TOKENIZER,
    compute_metrics=compute_metrics,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /home/jovyan/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range

In [33]:
# in case the process gets interrupted, it would be nice to keep some results
import json
import logging
import time
from itertools import product

logfile = 'hyperparameter_search.log'
logging.basicConfig(level=logging.INFO,
                    filename=logfile,
                    format='%(asctime)s -- %(message)s')

param_grid = {
    'learning_rate': [2e-5, 3e-5, 5e-5], # Range suggested by Devlin et al.
    'num_train_epochs': [2, 3, 4], # Range suggested by Devlin et al.
    'batch_size': [16, 32],
}

logging.info('New Hyperparameter Search')
logging.info(json.dumps(param_grid))

# Since we have 2 settings with 3 values each, we will finetune 9 times
cartesian_product = [dict(zip(param_grid.keys(), values)) for 
                     values in product(*param_grid.values())]

for i, params in enumerate(cartesian_product):
    for k, v in params.items():
        setattr(trainer.args, k, v)

    start = time.time()
    trainer.train()
    time_elapsed = time.time() - start

    preds = trainer.predict(datadict['dev'])
    r2 = r2_score(preds.label_ids, preds.predictions)

    print(', '.join([f'{k}: {v}' for k, v in params.items()]), end=' ')
    print(f'--> R-squared: {r2:.3f}')
    print(f'Time elapsed: {time_elapsed:.0f} seconds')

    params['trial'] = i
    params['score'] = r2
    params['seconds'] = time_elapsed

    logging.info(json.dumps(params))

loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /home/jovyan/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden

Epoch,Training Loss,Validation Loss


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: learner_id, writing_id, text, cefr, grade, l1. If learner_id, writing_id, text, cefr, grade, l1 are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 17668
  Batch size = 8


ValueError: Classification metrics can't handle a mix of multiclass and continuous-multioutput targets